In [99]:
import os, sys, json, time
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER   = C["ticker"]; START = C["start"]; END = C["end"]; INTERVAL = C["interval"]
HORIZON  = int(C["horizon"]); LOOKBACK = int(C["lookback"])
BATCH    = int(C["batch"]);   EPOCHS   = int(C["epochs"])
SEED     = int(C.get("seed", 42))

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR   = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_lstm")
RUN_DIR.mkdir(parents=True, exist_ok=True)   # <- hinzufügen
print("RUN_DIR:", RUN_DIR)                   # optional

TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}.csv"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, matthews_corrcoef, average_precision_score
import joblib

np.random.seed(SEED)
tf.random.set_seed(SEED)


RUN_DIR: ..\results\2025-10-03_13-48-57_lstm


In [100]:
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=[0]).sort_index()

exp = {"open","high","low","close","volume","logret_1d","target"}
missing = exp - set(df.columns)
assert not missing, f"Fehlende Spalten: {missing}"
assert not df.index.has_duplicates
assert (df["close"] > 0).all()
assert df.notna().all().all()


In [101]:
# === 3) Features / Ziel wählen (stationär & robust) ===
import os, yaml

OHLCV = {"open","high","low","close","volume"}

# Kandidaten aus DF (alles außer OHLCV + target)
candidates_from_df = [c for c in df.columns if c not in OHLCV | {"target"}]

# YAML laden (falls vorhanden) und schneiden
features_yaml = "../data/features_v1.yml"
yaml_feats = []
if os.path.exists(features_yaml):
    with open(features_yaml, "r") as f:
        meta = yaml.safe_load(f) or {}
    yaml_feats = list(meta.get("features", []))

# Level-Features blocken (driften stark)
BLOCKED = {"sma_10", "sma_20"}

# Priorität: YAML ∩ DF, danach DF-Fallback; anschließend Blockliste anwenden
FEATURES = [c for c in (yaml_feats or candidates_from_df) if c in candidates_from_df and c not in BLOCKED]

# Sicherheit: nur numerische Spalten
FEATURES = [c for c in FEATURES if pd.api.types.is_numeric_dtype(df[c])]

assert len(FEATURES) > 0, f"Keine nutzbaren Features gefunden. Im CSV sind Kandidaten: {candidates_from_df}"
print("FEATURES (final):", FEATURES)

TARGET = "target"
X = df[FEATURES].copy()
y = df[TARGET].astype(int).copy()


FEATURES (final): ['logret_1d', 'sma_diff']


In [102]:
# === 4) Chronologische Splits (70/15/15) ===
n = len(df)
n_train = int(n * 0.70)
n_val   = int(n * 0.15)
n_test  = n - n_train - n_val

train_idx = slice(0, n_train)
val_idx   = slice(n_train, n_train + n_val)
test_idx  = slice(n_train + n_val, n)

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

print(f"Split sizes → train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")

Split sizes → train 2381, val 510, test 511


In [103]:
print("X_train shape/check:", X_train.shape, " | cols:", list(X_train.columns))


X_train shape/check: (2381, 2)  | cols: ['logret_1d', 'sma_diff']


In [104]:
# === 5) Scaler nur auf TRAIN fitten ===
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),       index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),      index=X_test.index,  columns=FEATURES)

# Scaler speichern (für spätere Runs/Inference)
import joblib, io
joblib.dump(scaler, RUN_DIR / "scaler.joblib")

['..\\results\\2025-10-03_13-48-57_lstm\\scaler.joblib']

In [105]:
def drift_summary(Xa: pd.DataFrame, Xb: pd.DataFrame):
    out = []
    for c in Xa.columns:
        m1, s1 = Xa[c].mean(), Xa[c].std(ddof=1)
        m2, s2 = Xb[c].mean(), Xb[c].std(ddof=1)
        ratio_std = float((s2 + 1e-9) / (s1 + 1e-9))
        diff_mean = float(m2 - m1)
        out.append({"feature": c, "mean_diff": diff_mean, "std_ratio": ratio_std})
    return pd.DataFrame(out).sort_values("std_ratio", ascending=False)

drift_df = drift_summary(X_train_s, X_test_s)
drift_df.to_csv(RUN_DIR / "drift_train_vs_test.csv", index=False)
print(drift_df.head())


     feature  mean_diff  std_ratio
0  logret_1d  -0.021755   0.957276
1   sma_diff  -0.125661   0.915388


In [106]:
# Warn-/Abbruchschwellen gegen Train→Test-Shift
bad = drift_df[(drift_df["std_ratio"] < 0.85) | (drift_df["mean_diff"].abs() > 1.0)]
if not bad.empty:
    print("\n[WARN] Starker Feature-Shift erkannt:\n", bad)
    # Optional hart abbrechen:
    # raise RuntimeError("Zu starker Drift in obigen Features – bitte Feature-Set stationär halten.")

In [107]:
# === 6) Windowing: Sequenzen der Länge LOOKBACK → Label am Endzeitpunkt ===
def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys = [], []
    for i in range(lookback-1, n):
        xs.append(X_values[i - lookback + 1 : i + 1])  # inkl. i
        ys.append(y_values[i])                          # Label für Zeitpunkt i (Up/Down für i->i+H)
    return np.stack(xs, axis=0), np.array(ys)

Xtr_win, ytr = make_windows(X_train_s, y_train, LOOKBACK)
Xva_win, yva = make_windows(X_val_s,   y_val,   LOOKBACK)
Xte_win, yte = make_windows(X_test_s,  y_test,  LOOKBACK)

print("Shapes:",
      "\n  train:", Xtr_win.shape, ytr.shape,
      "\n  val  :", Xva_win.shape, yva.shape,
      "\n  test :", Xte_win.shape, yte.shape)

Shapes: 
  train: (2322, 60, 2) (2322,) 
  val  : (451, 60, 2) (451,) 
  test : (452, 60, 2) (452,)


In [108]:
# === class_weight (optional) aus Trainingslabels berechnen ===
from collections import Counter
cw = None
counts = Counter(ytr.tolist())
if len(counts) == 2:
    total = sum(counts.values())
    # einfache Invers-Häufigkeit (normalisiert), robust bei leichter Schieflage
    cw = {0: total/(2*counts.get(0, 1)), 1: total/(2*counts.get(1, 1))}
print("class_weight:", cw)


class_weight: {0: 1.0554545454545454, 1: 0.9500818330605565}


In [109]:
# === 7) tf.data Pipelines ===
def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

ds_train = to_ds(Xtr_win, ytr, BATCH, shuffle=True)
ds_val   = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test  = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [110]:
# Basis-Rate im Training (für Output-Bias)
pos_rate_train = float(ytr.mean())
from math import log
def _logit(p): 
    eps = 1e-6
    p = min(max(p, eps), 1-eps)
    return log(p/(1-p))
output_bias_init = tf.keras.initializers.Constant(_logit(pos_rate_train))
print("pos_rate_train:", round(pos_rate_train,3))


pos_rate_train: 0.526


In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, matthews_corrcoef

logit = LogisticRegression(max_iter=200, n_jobs=None)
logit.fit(X_train_s.iloc[LOOKBACK-1:], y_train.iloc[LOOKBACK-1:])  # grob: letztes Fensterende
y_proba_lr = logit.predict_proba(X_test_s.iloc[LOOKBACK-1:])[:,1]
print(f"[Diag] y_proba range: {y_proba.min():.3f} .. {y_proba.max():.3f}, mean={y_proba.mean():.3f}")
print("LogReg AUROC:", round(roc_auc_score(y_test.iloc[LOOKBACK-1:], y_proba_lr), 3))
print("LogReg MCC@0.5:", round(matthews_corrcoef(y_test.iloc[LOOKBACK-1:], (y_proba_lr>=0.5).astype(int)), 3))


[Diag] y_proba range: 0.263 .. 0.613, mean=0.512
LogReg AUROC: 0.482
LogReg MCC@0.5: -0.018


In [112]:
# === Modell: GRU + Regularisierung ============================================
from tensorflow import keras
from tensorflow.keras import layers, regularizers

USE_FOCAL = bool(C.get("use_focal", True))
GAMMA     = float(C.get("focal_gamma", 1.5))
ALPHA     = float(C.get("focal_alpha", 0.5))

def focal_loss(gamma=2.0, alpha=0.25):
    def _loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        eps = 1e-7
        y_pred = tf.clip_by_value(y_pred, eps, 1-eps)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1-y_pred)
        w  = tf.where(tf.equal(y_true, 1), alpha, 1-alpha)
        return -tf.reduce_mean(w * tf.pow(1.0-pt, gamma) * tf.math.log(pt))
    return _loss

n_features = Xtr_win.shape[-1]

model = keras.Sequential([
    layers.Input(shape=(LOOKBACK, n_features)),
    layers.GRU(64, return_sequences=True, recurrent_dropout=0.10),
    layers.LayerNormalization(),
    layers.GRU(32, recurrent_dropout=0.10),
    layers.LayerNormalization(),
    layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-5)),
    layers.Dense(1, activation="sigmoid", bias_initializer=output_bias_init),
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-4),
    loss=focal_loss(GAMMA, ALPHA) if USE_FOCAL else keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.AUC(name="auc"),
        keras.metrics.AUC(name="auprc", curve="PR"),
        keras.metrics.BinaryAccuracy(name="acc"),
        keras.metrics.Precision(name="prec"),
        keras.metrics.Recall(name="rec"),
    ],
)

In [113]:
ckpt_path = RUN_DIR / "best.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(ckpt_path),
        monitor="val_auprc", mode="max", save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(
        monitor="val_auprc", mode="max", patience=12, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_auprc", mode="max", factor=0.5, patience=6, min_lr=1e-5, verbose=1),
]


In [114]:
history = model.fit(
    ds_train, validation_data=ds_val, epochs=EPOCHS,
    callbacks=callbacks, verbose=1, class_weight=cw
)

# Trainingskurve speichern
pd.DataFrame(history.history).to_csv(RUN_DIR / "history.csv", index=False)

Epoch 1/100
34/37 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.5323 - auc: 0.5153 - auprc: 0.5349 - loss: 0.1647 - prec: 0.5479 - rec: 0.7080
Epoch 1: val_auprc improved from None to 0.51487, saving model to ..\results\2025-10-03_13-48-57_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - acc: 0.5211 - auc: 0.5143 - auprc: 0.5323 - loss: 0.1409 - prec: 0.5370 - rec: 0.6530 - val_acc: 0.4878 - val_auc: 0.5001 - val_auprc: 0.5149 - val_loss: 0.1278 - val_prec: 0.4906 - val_rec: 0.3421 - learning_rate: 5.0000e-04
Epoch 2/100
36/37 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.5058 - auc: 0.4989 - auprc: 0.5226 - loss: 0.1297 - prec: 0.5284 - rec: 0.5827
Epoch 2: val_auprc improved from 0.51487 to 0.54146, saving model to ..\results\2025-10-03_13-48-57_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - acc: 0.5052 - auc: 0.5023 - auprc: 0.5272 - loss: 0.1283 - prec: 0.5264 - rec: 0.5949 - val_acc: 0.4900 - val_auc: 0.5055 - val_auprc: 0.5415 - val_loss: 0.1249 - val_prec: 0.4970 -

In [115]:
# env-info sauber mitschreiben
env_info = {
    "python": sys.version, "tensorflow": tf.__version__,
    "numpy": np.__version__, "pandas": pd.__version__,
    "seed": SEED, "lookback": LOOKBACK, "features": FEATURES,
    "batch": BATCH, "epochs": EPOCHS, "optimizer": "Adam(5e-4)",
    "loss": "Focal" if USE_FOCAL else "BCE",
    "metrics": ["acc","auc","auprc","prec","rec"],
}
with open(RUN_DIR / "env_info.json", "w") as f: json.dump(env_info, f, indent=2)
np.save(RUN_DIR / "scaler_mean.npy", scaler.mean_)
np.save(RUN_DIR / "scaler_scale.npy", scaler.scale_)

In [116]:
# === 11) Evaluate & Berichte ===
# Best Weights sind dank EarlyStopping bereits geladen
test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
print("Test metrics:", json.dumps(test_metrics, indent=2))

# Schwellenwert 0.5 (später kalibrierbar)
y_proba = model.predict(ds_test, verbose=0).ravel()
y_pred  = (y_proba >= 0.5).astype(int)

print("\nConfusion matrix (test):\n", confusion_matrix(yte, y_pred))
print("\nClassification report (test):\n", classification_report(yte, y_pred, digits=3))

Test metrics: {
  "acc": 0.5442478060722351,
  "auc": 0.5026698112487793,
  "auprc": 0.548076868057251,
  "loss": 0.12539570033550262,
  "prec": 0.555232584476471,
  "rec": 0.7827869057655334
}

Confusion matrix (test):
 [[ 55 153]
 [ 53 191]]

Classification report (test):
               precision    recall  f1-score   support

           0      0.509     0.264     0.348       208
           1      0.555     0.783     0.650       244

    accuracy                          0.544       452
   macro avg      0.532     0.524     0.499       452
weighted avg      0.534     0.544     0.511       452



In [117]:
# Val-basierte Schwelle (max MCC, Korridor) direkt in Block 3
from sklearn.metrics import matthews_corrcoef, roc_auc_score
import numpy as np

val_proba = model.predict(ds_val, verbose=0).ravel()

def choose_threshold(y_true, y_prob, bounds=(0.2, 0.8)):
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1
    for t in cand:
        yp = (y_prob >= t).astype(int)
        pr = yp.mean()
        if not (bounds[0] <= pr <= bounds[1]): 
            continue
        s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = s, float(t)
    return best_t

thr = choose_threshold(yva, val_proba, bounds=(0.2,0.8))
y_pred_thr = (y_proba >= thr).astype(int)

print(f"\n[Block3 quick] thr(val@maxMCC)={thr:.3f} | AUROC val/test={roc_auc_score(yva, val_proba):.3f}/{roc_auc_score(yte, y_proba):.3f}")
print("CM(test@thr):\n", confusion_matrix(yte, y_pred_thr))
print(classification_report(yte, y_pred_thr, digits=3))



[Block3 quick] thr(val@maxMCC)=0.521 | AUROC val/test=0.507/0.503
CM(test@thr):
 [[100 108]
 [118 126]]
              precision    recall  f1-score   support

           0      0.459     0.481     0.469       208
           1      0.538     0.516     0.527       244

    accuracy                          0.500       452
   macro avg      0.499     0.499     0.498       452
weighted avg      0.502     0.500     0.501       452



In [118]:
# --- Diagnose der Probabilitäten ---
import numpy as np
from sklearn.metrics import roc_auc_score

print("Proba stats  (test): min=", float(y_proba.min()), 
      "max=", float(y_proba.max()), "mean=", float(y_proba.mean()))

# AUC auf VAL & TEST (Ranking-Qualität, unabhängig vom Threshold)
val_proba = model.predict(ds_val, verbose=0).ravel()
print("AUROC val/test:", 
      round(roc_auc_score(yva, val_proba), 3), "/", 
      round(roc_auc_score(yte, y_proba), 3))

# Quick check: Ist das Signal invertiert?
if roc_auc_score(yva, val_proba) < 0.5:
    print("⚠️ AUROC < 0.5 auf VAL → Versuch: invertiere Scores (1-p)")
    y_proba_inverted = 1.0 - y_proba
    from sklearn.metrics import classification_report, confusion_matrix
    y_pred_inv = (y_proba_inverted >= 0.5).astype(int)
    print("Confusion (inv, thr=0.5):\n", confusion_matrix(yte, y_pred_inv))
    print("Report (inv):\n", classification_report(yte, y_pred_inv, digits=3))


Proba stats  (test): min= 0.2409822791814804 max= 0.646703839302063 mean= 0.5228776931762695
AUROC val/test: 0.507 / 0.503


In [119]:
# === Extra Test-Metriken ===
bal_acc = balanced_accuracy_score(yte, y_pred)
mcc = matthews_corrcoef(yte, y_pred)
auprc_test = average_precision_score(yte, y_proba)  # probabilistische PR-Qualität

extra = {
    "balanced_accuracy": float(bal_acc),
    "mcc": float(mcc),
    "auprc": float(auprc_test)
}
print("Extra test metrics:", json.dumps(extra, indent=2))

# persistieren
with open(RUN_DIR / "extra_test_metrics.json", "w") as f:
    json.dump(extra, f, indent=2)


Extra test metrics: {
  "balanced_accuracy": 0.5236049810844893,
  "mcc": 0.05517840392405117,
  "auprc": 0.5503723401714022
}


In [120]:
# === 12) Artefakte sichern ===
# Keras-Format (SavedModel) + Gewichte
model.save(RUN_DIR / "model.keras")
np.save(RUN_DIR / "y_test.npy", yte)
np.save(RUN_DIR / "y_proba.npy", y_proba)
with open(RUN_DIR / "config.json", "w") as f:
    json.dump({
        "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
        "horizon": HORIZON, "lookback": LOOKBACK, "features": FEATURES,
        "scaler": "StandardScaler", "seed": SEED, "batch": BATCH, "epochs": EPOCHS
    }, f, indent=2)
print(f"\nArtefakte gespeichert in: {RUN_DIR}")


Artefakte gespeichert in: ..\results\2025-10-03_13-48-57_lstm
